# NN by Hand

In [1]:
import numpy as np
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse

We're going to make a very simple neural network by hand.

Let's imagine that we have just a five-neuron network: two input neurons, two hidden-layer neurons, and a single output neuron.

We'll start by giving ourselves a regression problem.

![example](./images/one-hidden-layer-simplest-neural-network.jpg)

In [2]:
X, y = make_regression(n_features=2,
                       n_informative=1,
                       noise=100,
                       random_state=42)

Let's first check to see how a vanilla linear regression model does:

In [3]:
X[:5]

array([[-1.41537074, -0.42064532],
       [ 0.52194157,  0.29698467],
       [-0.88951443, -0.81581028],
       [-0.88385744,  0.15372511],
       [ 0.73846658,  0.17136828]])

In [4]:
simple_lr = LinearRegression().fit(X, y)

simple_lr.score(X, y)

0.4144461358278726

In [5]:
mse(y, simple_lr.predict(X), squared=False)

101.32961290191164

## Input Neurons

We have just two features, so we'll have two input neurons.

In [6]:
X.shape

(100, 2)

In [7]:
n_samples = X.shape[0]
n_features = X.shape[1]

# Initial inputs
input1 = X[:, 0].reshape(-1, 1)
input2 = X[:, 1].reshape(-1, 1)

In [9]:
input1.shape

(100, 1)

## Weights for the Connections Between the Input Neurons and the Hidden Neurons

We'll just start by setting our weights randomly. The idea will be that we'll be able to use gradient descent to improve their values during network training.

In [10]:
# Four weights to optimize for between input
# and hidden layers

# For simplicity let's assume biases of 0
# throughout

np.random.seed(42)

in_hid_weights1 = np.random.rand(2)
in_hid_weights2 = np.random.rand(2)

In [11]:
in_hid_weights1

array([0.37454012, 0.95071431])

## Hidden Neurons

Each neuron in the hidden layer will take the two inputs and multiply them by a unique set of weights:

In [12]:
in1_to_hid = (np.sum(np.product(np.array([in_hid_weights1, input1], dtype=object), axis=0), axis=1)).reshape(-1, 1)
in2_to_hid = (np.sum(np.product(np.array([in_hid_weights2, input2], dtype=object), axis=0), axis=1)).reshape(-1, 1)

In [13]:
in1_to_hid

array([[-1.87572634],
       [ 0.69170537],
       [-1.17883293],
       [-1.17133598],
       [ 0.9786561 ],
       [-0.35073764],
       [ 1.51453099],
       [ 0.47894113],
       [ 1.07680344],
       [-0.29614505],
       [-1.46617532],
       [-1.12221712],
       [-0.79599908],
       [ 0.00677663],
       [-0.3103128 ],
       [-2.13032427],
       [ 0.12865245],
       [ 0.42949371],
       [-0.51964307],
       [-0.09543173],
       [-0.61414635],
       [ 2.49967621],
       [ 2.09285877],
       [ 0.13206341],
       [ 1.07811729],
       [-1.57878023],
       [ 0.77773496],
       [-1.76018444],
       [-0.1532634 ],
       [ 0.65827263],
       [-0.29112114],
       [-2.05502357],
       [-2.28595499],
       [ 2.47262593],
       [-0.10217938],
       [-2.54286004],
       [ 0.8583521 ],
       [-0.89709388],
       [ 1.53543392],
       [ 0.12160638],
       [-0.32520169],
       [ 0.32066157],
       [-3.47182879],
       [ 0.36668571],
       [ 1.21314071],
       [-1

For simplicity let's assume a **linear activation function**, $f(x)=x$, in the hidden nodes.

## Weights for the Connections Between the Hidden Neurons and the Output Neuron

Again we'll just set our weights randomly. Here there will be two weights: one governing the connection between each hidden neuron and the output neuron.

In [14]:
np.random.seed(42)

hid_out_weights = np.random.rand(2)

In [15]:
hid_out_weights

array([0.37454012, 0.95071431])

## Output Neuron

Now we need to take the contribution from each hidden neuron and create a linear sum with the predetermined weights, just as above in the hidden neurons.

In [16]:
joint_to_out = np.hstack((in1_to_hid, in2_to_hid))

In [17]:
output = (np.sum(np.product([hid_out_weights, joint_to_out],dtype=object, axis=0), axis=1)).reshape(-1, 1)

C:\Users\david\anaconda3\envs\learn-env\lib\site-packages\numpy\core\fromnumeric.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [18]:
output[:5]

array([[-1.23468067],
       [ 0.6347779 ],
       [-1.47357759],
       [-0.24423925],
       [ 0.5833389 ]])

Again we'll assume a linear activation.

In [19]:
X[:5]

array([[-1.41537074, -0.42064532],
       [ 0.52194157,  0.29698467],
       [-0.88951443, -0.81581028],
       [-0.88385744,  0.15372511],
       [ 0.73846658,  0.17136828]])

## Error Calculation

Now we can compute a measure of error:

In [22]:
output = output.flatten()
mse(y, output, squared=False)

132.0854176645698

### Backpropagation

Now then: How do we make corrections to our weights to improve our model's performance? Our network looks like this:

![nn](images/SimpleNN.png)

Clearly our output is a function of these six weights. But what function, exactly?

- In the top hidden node we construct: <br/> $H_1 = w_1X_1 + w_3X_2$ <br/> and in the bottom node we construct: <br/> $H_2 = w_2X_1 + w_4X_2$.
<br/> <br/>
- In the output node we construct: <br/> $\hat{y} = w_5H_1 + w_6H_2$ <br/> i.e. <br/> $\hat{y} = w_5(w_1X_1 + w_3X_2) + w_6(w_2X_1 + w_4X_2)$ <br/> or <br/> $\hat{y} = X_1(w_1w_5 + w_2w_6) + X_2(w_3w_5 + w_4w_6)$.

#### Loss Function

Our loss function (let's assume) is just $\mathcal{L} = \Sigma\left(y - \hat{y}\right)^2$. What are the partial derivatives of this function?

We have $\mathcal{L} = \Sigma\left(y - [X_1(w_1w_5 + w_2w_6) + X_2(w_3w_5 + w_4w_6)]\right)^2$.

Therefore:

- $\frac{\partial\mathcal{L}}{\partial w_1} = -2w_5X_1\Sigma\left(y - [X_1(w_1w_5 + w_2w_6) + X_2(w_3w_5 + w_4w_6)]\right)$

- $\frac{\partial\mathcal{L}}{\partial w_2} = -2w_6X_1\Sigma\left(y - [X_1(w_1w_5 + w_2w_6) + X_2(w_3w_5 + w_4w_6)]\right)$

- $\frac{\partial\mathcal{L}}{\partial w_3} = -2w_5X_2\Sigma\left(y - [X_1(w_1w_5 + w_2w_6) + X_2(w_3w_5 + w_4w_6)]\right)$

- $\frac{\partial\mathcal{L}}{\partial w_4} = -2w_6X_2\Sigma\left(y - [X_1(w_1w_5 + w_2w_6) + X_2(w_3w_5 + w_4w_6)]\right)$

- $\frac{\partial\mathcal{L}}{\partial w_5} = -2(w_1X_1 + w_3X_2)\Sigma\left(y - [X_1(w_1w_5 + w_2w_6) + X_2(w_3w_5 + w_4w_6)]\right)$

- $\frac{\partial\mathcal{L}}{\partial w_6} = -2(w_2X_1 + w_4X_2)\Sigma\left(y - [X_1(w_1w_5 + w_2w_6) + X_2(w_3w_5 + w_4w_6)]\right)$

So the goal now should just be to nudge each of our weights in (the opposites of) these directions.

Let's first isolate each of these six weights:

In [25]:
w1 = in_hid_weights1[0]
w2 = in_hid_weights1[1]
w3 = in_hid_weights2[0]
w4 = in_hid_weights2[1]
w5 = hid_out_weights[0]
w6 = hid_out_weights[1]

Let's build an SGD function that will adjust weights after each training sample runs through the network.

In [26]:
def stoch_grad_desc(pred1=input1, pred2=input2, y=y,
              w1=w1, w2=w2, w3=w3, w4=w4,
              w5=w5, w6=w6, times_thru=1, lr=1e-4):

    for k in range(times_thru):
    
        for j in range(pred1.shape[0]):
            
            in1_val = pred1[j]
            in2_val = pred2[j]
            
            # This is stochastic gradient descent since we are updating
            # our weights *after each data point* goes through the network.
            
            sum_ = y[j] - in1_val*(w1*w5 + w2*w6) - in2_val*(w3*w5 + w4*w6)

            w5 += lr*(w1*in1_val + w3*in2_val)*sum_

            w6 += lr*(w2*in1_val + w4*in2_val)*sum_

            w1 += lr*w5*in1_val*sum_

            w2 += lr*w6*in1_val*sum_

            w3 += lr*w5*in2_val*sum_

            w4 += lr*w6*in2_val*sum_
            
            output = pred1*(w1*w5 + w2*w6) + pred2*(w3*w5 + w4*w6)
            
            if j == 0 and k == 0:
                print(f"""
                After a single data point our RMSE is {np.sqrt(mse(y, output.flatten()))}
                """)
            
        print(f"""
                After {k+1} epochs our RMSE is {np.sqrt(mse(y, output.flatten()))}
        """)
        
    return w1, w2, w3, w4, w5, w6

In [27]:
stoch_grad_desc(times_thru=5)


                After a single data point our RMSE is 131.80677685986092
                

                After 1 epochs our RMSE is 129.8450062254458
        

                After 2 epochs our RMSE is 123.33196509825842
        

                After 3 epochs our RMSE is 111.44619152943271
        

                After 4 epochs our RMSE is 104.25314828273383
        

                After 5 epochs our RMSE is 102.76034914846261
        


(array([3.74787473]),
 array([8.65770675]),
 array([0.88442709]),
 array([0.92008411]),
 array([3.63496368]),
 array([8.36568275]))

## Easy Regression Problem

Our error has decreased with more epochs, but to illustrate the full power of our network let's see if it can find the right coefficients for an "easy" problem where there is a strong correlation between both of two input features and the target.

In [28]:
# Setting up the problem

X_easy, y_easy = make_regression(n_features=2, n_informative=2, random_state=42)

In [29]:
# Again defining weights randomly. We'll be using the same network, so we need
# six weights.

np.random.seed(42)

w_easy = np.random.rand(6)

In [30]:
# Applying our stoch_grad_desc() function

final_weights = stoch_grad_desc(pred1=X_easy[:, 0], pred2=X_easy[:, 1], y=y_easy,
         w1=w_easy[0], w2=w_easy[1], w3=w_easy[2],
         w4=w_easy[3], w5=w_easy[4], w6=w_easy[5],
         lr=4e-3, times_thru=3)
final_weights


                After a single data point our RMSE is 105.2827433708648
                

                After 1 epochs our RMSE is 0.0003058349135338318
        

                After 2 epochs our RMSE is 5.040642635748519e-09
        

                After 3 epochs our RMSE is 1.2706912599150398e-13
        


(6.556340807199361,
 10.75555705679945,
 6.914105827696129,
 7.997420739574159,
 4.334717820507707,
 5.515048597227902)

Wow! Look how small our error is after just three epochs!

### Comparing with `LinearRegression()`

We can translate these final weight values into $\beta_1$ and $\beta_2$ for a traditional linear regression $\hat{y} = \beta_1X_1 + \beta_2X_2$.

Above we calculated $\hat{y} = X_1(w_1w_5 + w_2w_6) + X_2(w_3w_5 + w_4w_6)$.

Thus we have:

- $\beta_1 = w_1w_5 + w_2w_6$, and
- $\beta_2 = w_3w_5 + w_4w_6$.

Plugging these in for our final calculated weights we have:

In [31]:
beta1 = final_weights[0]*final_weights[4] + final_weights[1]*final_weights[5]
beta2 = final_weights[2]*final_weights[4] + final_weights[3]*final_weights[5]
print(f"Our mini-NN found coefficients of {beta1} and {beta2}.")

Our mini-NN found coefficients of 87.73730719279541 and 74.07686177542038.


Let's compare these numbers with the results of `LinearRegression()`:

In [32]:
LinearRegression(fit_intercept=False).fit(X_easy, y_easy).coef_

array([87.73730719, 74.07686178])

Our little neural network, which has:
- only five neurons;
- randomly generated initial weights;
- only linear activation functions; and
- all biases set to 0

can do the same job as a linear regression after just a couple of epochs!